<a href="https://colab.research.google.com/github/branndonm1/branndonm1/blob/main/BinomialPricingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introductory Theory


Suppose the value of a particular stock $S$ is a random process. Say it has initial value  $S_0$  and its value at a later time depends on a coin flip. If we achieve a heads then the value at the later time is  $$S_1(H)=uS_0$$ where  $u\in (1,\infty)$. Similarly if we get tails $S_1(T)=dS_0$ where $d\in[0,1]$. We don't assume the coin flip is fair so we have to assign the probability of getting heads or tails but we will select this later on.

We'd like know what these up and down factors are and what the sucess probability of getting a head should be. This way we can predict the chances that a particular stock should go up and by how much its value should rise.

If our model predicts a stock should go up, then it would be smart of us to obtain some capital to buy shares of the stock. We may want to take out a loan of value $B_0$ with constant interest rate  $r_0>0$. The value of the loan is deterministic so that at a later till we will have to pay the bank back $$B_1=(1+r)B_0.$$

Now suppose an portfolio of investments has initial value $X_0$ and takes the following investment strategy: "buy  $\Delta_0$  shares of stock $S$ and invest the rest at the bank". The value of the portfolio at the later time will be
$$(*) \hspace{.5cm} X_1(\omega)=(1+r)(X_0−\Delta_0S_0)+\Delta_0S_1(\omega).$$
Note that the value depends on whether the outcome of the coin flip was heads of tails.

Assuming no market frictions, and assuming that no individual can take advantage of the market (arbitrage), it can be shown that the parameters must satisfy the inequality  $$u>1+r>d.$$ Moreover, the success probability of head from before must be  $$p=\frac{1+r−d}{u−d}$$ which implicitly tells us the tails probability must be  $q=1−p=\frac{u−(1+r)}{u−d}.$

A derivative security $V$ is an asset whos value depends on a underlying stock $S$. A European Put options (EPO) is a particular option that allows holders to sell at a set "strike price" $K$ at some "expirations time" $T$ regardless of the price of the underlying stock $S$ at time $T$. If one "exercises" the put at time $T$ and the underlying stock has value less than $K$, one can make a profit of $K-S_T$. If the stock is valued higher than the stock then one simplify does not exercise the put and the EPO thus has a value of 0. In summary, at the experation time the value of the Euro put is $$(**) \hspace{.5cm} V_T(\omega) =\max\{0,K−S_T(\omega)\}.$$ Note that $V_T$ implicitly depends on the coin flip as well.

Now suppose we have the European Put $V$ and while we may know its value $V_T$ at the expiration time (see equation $(**))$, it is not clear how to define a fair price $V_0$ for the EPO at initial time. In theory one can create a "replicating portfolio" $X$ made up of only the underlying stock $S$ by carefully buying the number of shares of stock $S$ at initial time that will evolve to will match the payout of a Euro Put $V$ at expiration time. In order for $V_1(\omega) = X_1(\omega)$ we must have the number of share bought $\Delta_0 = \frac{V_1(H)-V_1(T)}{S_1(H)-S_1(T)}$ and we can then plug this into $(*)$ to get the value of the portfolio at initial time $$(***) \hspace{.5cm} X_0=\frac{1}{1+r}E[X_1]=\frac{1}{1+r}[pX(H)+qX(T)].$$ and we define this to be the fair price of the Euro Put $V$ at initial time $$V_0 := X_0.$$

The key idea here is that if we believe there is a self replicating portfolio $X$ that can match the performance of the Euro Put $V$ we can use the backwards induction formula $(***)$ to find the initial value of the portfolio which by definition is the fair initial price of the Euro Put $V$.

Assuming that the market is fair and arbitrage free, the predicted price $V_0$ should match the current listed price of the particular put option with strike price $K$. This assume that we know the value of the put at expiration time, but that means we must know the value of the stock at expiration time. We don't know this exactly, but we can guess the up and down factors to get a good estimate. Using this guess we can do back propogation to estimate the fair price and compare it to the actual listen price. We can keep guessing until we get an estimated fair price that is close to the actual listed price. Of course guessing is not the most efficient way to find the "best fit" up and down factors. We will use a mathematical techniques to solve this problem.

# Installing Packages and Importing Libraries

In [ ]:
!pip install yfinance #install yfinance for API

import numpy as np #import numpy for handy array and math stuff
import matplotlib.pyplot as plt #for plotting data
import pandas as pd #for stats/excel type stuff
import seaborn as sns #for statistical plots
import yfinance as yf #api to get stock data from yahoo finance
import scipy #for math stuff
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Getting Data

In [ ]:
#pick stocks
tkr="AAPL" #get tickers for desired stock
stk = yf.Ticker(tkr) #feed desired stock

#get stock data
stock_data = yf.download(tkr, start="2000-01-01", end="2022-08-01") #download stock data between given dates

#get options data
minExpData = min(stk.options) #take options with closest expiration date
opt=stk.option_chain(minExpData) #gets options data for that particular expiration date
strikes = opt.puts['strike'] #gets list of all possible strike prices for the put options with that expiration date
prices = opt.puts['lastPrice'] #gets current price of put option



[*********************100%***********************]  1 of 1 completed


# New Section

In [ ]:
#initialize model params
s_not = stock_data['Open'].tail(1).values[0] #open price
u = 1+(stock_data['High'].tail(1).values[0] - stock_data['Open'].tail(1).values[0])/stock_data['Open'].tail(1).values[0] #(high-open)/open
d = 1-(stock_data['Open'].tail(1).values[0] - stock_data['Low'].tail(1).values[0])/stock_data['Open'].tail(1).values[0] #(open-low)/open
r = (u+d-2)/2 #picked so that u>r+1>d

print("S_0="+str(s_not))
print("u="+str(u))
print("d="+str(d))
print("r="+str(r))

S_0=161.24000549316406
u=1.0148226203685522
d=0.9892085993929228
r=0.0020156098807375677


In [ ]:

def grid_optimize(u_s, d_s, r_s):
  minError = 1000
  best_params = [1,1,1]
  for u in u_s:
    for d in d_s:
      for r in r_s:
        layers=100
        k=[strikes.values[i] for i in range(strikes.size)]
        p=(1+r-d)/(u-d)
        q=(u-(1+r))/(u-d)

        #estimates final stock prices at expiration given s_0, u, and, d
        final_stock = [pow(u,i)*pow(d,layers-i)*s_not for i in range(layers+1)]

        #given final stock prices back propogates to find option fair price today
        fair_prices = []

        for m in range(len(k)):
          option_tree = [[0 for j in range(layers-i+1)] for i in range(layers+1)]
          option_tree[0] = [max(k[m]-final_stock[i], 0) for i in range(len(final_stock))]
          for i in range(1,layers+1):
            for j in range(len(option_tree[i])):
              option_tree[i][j] = (p*option_tree[i-1][j]+q*option_tree[i-1][j+1])/(1+r)
          fair_prices.append(option_tree[layers][0])

        if mean_squared_error(fair_prices, prices) < minError:
          best_params = [u,d,r]
          minError = mean_squared_error(fair_prices, prices)
  return minError, best_params


# optimization step


In [ ]:
#grid search
u_s = [i+.01 for i in range(1,10)]
d_s = [1/i-.01 for i in range(1,10)]
r_s = [1/(5*i) for i in range(1,20)]

print("The min error and best params are:",grid_optimize(u_s, d_s, r_s))


The min error and best params are: (359.19025841826914, [3.01, 0.10111111111111111, 0.010526315789473684])


In [ ]:
print(optimize([u], [d], [r]))

Current params: [1.0148226203685522, 0.9892085993929228, 0.0020156098807375677]
Current error: 1000
(302.0479127694791, [1.0148226203685522, 0.9892085993929228, 0.0020156098807375677])
